In [2]:
import pandas as pd
import pyodbc
import pgeocode
import useful_functions as use

In [6]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [3]:
nulls = pd.read_csv('../../Data/DEA/NULL_CAT_CD_ID.csv')

In [4]:
QUERY = \
        """
        SELECT DISTINCT 
        M.ME, E.ENTITY_ID 
        FROM 
        AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E 
        WHERE 
        M.PARTY_ID=E.PARTY_ID
        """
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
ENTITY_TO_ME = pd.read_sql(con=AMAEDW, sql=QUERY)

In [5]:
nulls['ME'] = use.fix_me(nulls.ME)
nulls = pd.merge(nulls, ENTITY_TO_ME, on='ME')

In [3]:
ONE = pd.read_csv('../../Data/DEA/DEA_Only.csv')
TWO = pd.read_csv('../../Data/DEA/DEA_Only_AIMS.csv')
triangulate = pd.read_csv('../../Data/DEA/Triangulating.csv')

In [5]:
TWO.enttity_id

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC,ENTITY_ID_x,entity_id,comm_id,src_cat_code,ENTITY_ID_y
0,1892959,106930210,16561405,NaN,1215 7th St SE Ste 110,NaN,NaN,Decatur,AL,35601,2005-12-16 20:35:36.375262,9806586,Office,1333770,1333770,9806586,DEA,1333770
1,1888621,102550404,25638874,NaN,P O Box 1460,230 Hospital Dr,E R Physician,Vernon,AL,35592,2015-03-19 21:11:52.503690,19912709,Office,1244694,1244694,19912709,DEA,1244694
2,1892055,102801768,23568660,NaN,501 20th St Ste 205,Ste 205,Tennessee Internal Medicine,Knoxville,TN,37916,2013-11-14 21:23:06.070661,19056281,Office,1275738,1275738,19056281,DEA,1275738
3,1904346,102810899,16462443,NaN,203 Avalon Ave,Ste 290,Shoals Professional Bldg,Muscle Shoals,AL,35661,2014-06-04 21:15:29.556884,7984095,Office,1276947,1276947,7984095,DEA,1276947
4,1892066,102810961,20307531,NaN,910 Royce St,Medi-Weightloss Clinics,NaN,Pensacola,FL,32503,2009-12-24 21:00:40.770256,13737076,Office,1277025,1277025,13737076,DEA,1277025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119262,10855048,2878141558,19770188,NaN,651 E Prescott Rd,Smoky Hill Family Medicine Residency Pro,NaN,Salina,KS,67401,2016-10-06 21:31:00.420607,12045959,Office,48200408,48200408,12045959,DEA,48200408
119263,10888083,2576151376,23390994,NaN,50 N Perry St,McLaren Oakland,NaN,Pontiac,MI,48342,2016-09-29 21:31:09.931961,18817816,Office,48220740,48220740,18817816,DEA,48220740
119264,10888088,2576151422,23390994,NaN,50 N Perry St,McLaren Oakland,NaN,Pontiac,MI,48342,2016-09-29 21:31:09.933636,18817816,Office,48220805,48220805,18817816,DEA,48220805
119265,10900110,3576110595,10598363,NaN,327 Beach 19th St,NaN,NaN,Far Rockaway,NY,11691,2016-09-22 21:13:10.607963,6644915,Office,48223054,48223054,6644915,DEA,48223054


In [20]:
query_3 = \
    f'''
    SELECT
    C.ENTITY_ID,
    C.COMM_ID,
    C.SRC_CAT_CODE,
    U.COMM_TYPE,
    U.COMM_USAGE
    FROM
    informix.ENTITY_COMM_AT C, informix.ENTITY_COMM_USG_AT U
    WHERE
    U.ENTITY_ID = C.ENTITY_ID
    AND
    U.COMM_ID = C.COMM_ID
    AND
    U.COMM_CAT = 'A'
    AND
    U.END_DT IS NOT NULL
    AND
    C.END_DT IS NOT NULL
    AND
    C.SRC_CAT_CODE <> 'DEA       '
    ;
    '''
polo_addresses = pd.read_sql(con=informix, sql=query_3)
polo_addresses.head()

,entity_id,comm_id,src_cat_code,comm_type,comm_usage
0,3027527,875660,UNKNOWN,OF,PO
1,3027579,875691,ACS,OF,PP
2,3027605,875701,UNKNOWN,OF,PP
3,3027618,875706,OBSOLETE,OF,PO
4,3027644,875716,ROSTER,OF,PO


In [21]:
len(polo_addresses)

13099741

In [22]:
polo_addresses.to_csv('../../Data/DEA/null_NOT_DEAs.csv', index=False)

In [23]:
pd.read_csv('../../Data/DEA/null_NOT_DEAs.csv')

,entity_id,comm_id,src_cat_code,comm_type,comm_usage
0,3027527,875660,UNKNOWN,OF,PO
1,3027579,875691,ACS,OF,PP
2,3027605,875701,UNKNOWN,OF,PP
3,3027618,875706,OBSOLETE,OF,PO
4,3027644,875716,ROSTER,OF,PO
...,...,...,...,...,...
13099736,48984971,28280221,AMC,OF,AMC
13099737,46767639,22319636,AMC,HM,PP
13099738,69964778,14145736,AMC,OF,PP
13099739,47780482,25027118,PHONE-CALL,HM,PP


In [10]:
polo_addresses.src_cat_code.unique()

array(['ACS       ', None, 'DEA       ', 'WHITE-MAIL', 'PPA       ',
       'NPI       ', 'LICENSURE ', 'UNKNOWN   ', 'ROSTER    ',
       'AMA-ORG   ', 'PHONE-CALL', 'MFLOAD    ', 'E-MAIL    ',
       'OLDCC     ', 'FAST-TRACK', 'MBSHP-MAIL', 'GROUP     ',
       'LIST-HOUSE', 'INTERNET  ', 'SCHL-HOSP ', 'OTHER     ',
       'WEB       ', 'PUBS      ', 'RETURNED  ', 'PRFSOL    ',
       'NCOA      ', 'ACXIOMNCOA', 'ADMIT-HOS ', 'AFFIL-GRP ',
       'LOCK_BOX  ', 'MBSHP-WEB ', 'ADVR      ', 'PHNSURV   ',
       'ACXIOMLODE', 'MBSHP-OTHR', 'MRKT-RSRCH', 'ADDR-VER  ',
       'PPS       ', 'WEBSURV   ', 'OBSOLETE  ', 'MEDEC     ',
       'AMC       ', 'FEDERATION', 'USC-OUTBND', 'CME-REG   ',
       'REQ-CARDS ', 'MBSHP-PURL', 'PPMA      ', 'JOIN-RENEW',
       'MBSHP-IPM ', 'WEBSRCH   ', 'WEBPHYC   ', 'RES-TIPON ',
       'MBSHP-PHON', 'YELLOW    ', 'ACSIMB-MMX', 'POLO-MODEL',
       'POLO-PPD  ', 'NAT-DOWN  ', 'CRED-ADDR ', 'ACXIOMPLUS',
       'ACXIOM    ', 'COA-PS    ', 'PAS       ', 

In [13]:
ENTITY_TO_ME['entity_id'] = [int(x) for x in ENTITY_TO_ME.ENTITY_ID]

In [14]:
polo_addresses = pd.merge(polo_addresses, ENTITY_TO_ME, on='entity_id')

In [29]:
informix.close()

In [7]:
ppd_2 = pd.read_csv('../../Data/PPD/ppd_analysis_file.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,18,24,25,57,64,65,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
ppd_2.ME = use.fix_me(ppd_2.ME)

In [24]:
dpc = ppd_2[ppd_2.TOP_CD==20]

In [25]:
with_polos = ppd_2[~ppd_2.POLO_STATE.isna()]

In [29]:
with_polos_dpc = dpc[~dpc.POLO_STATE.isna()]

In [89]:
dpc[dpc.POLO_COMM_ID.isna()]

,ME,PARTY_ID,ENTITY_ID,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
85,00102001079,1849573,1235750,A,1,2.0,HITESH R PATEL MD,PATEL,HITESH,RAMAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,00102001192,1891442,1235841,A,1,2.0,MICHAEL P MCDOWELL MD,MCDOWELL,MICHAEL,P,...,NaN,NaN,NaN,NaN,MC DOWELL,P,MICHAEL,MC DOWELL,PATRICK,M
170,00102010167,1849855,1236920,A,1,2.0,SARAH LAUREN C HUGHEY MD,HUGHEY,SARAH LAUREN,CAMPBELL,...,NaN,NaN,NaN,NaN,CAMPBELL,LAUREN,SARAH,NaN,NaN,NaN
414,00102020995,2861216,15049264,A,1,2.0,RAFIQUE J MAHMOOD MD,MAHMOOD,RAFIQUE,JAHAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697,00102040678,3376959,21761346,A,1,2.0,STEVEN D BRIGHT MD,BRIGHT,STEVEN,DUSTIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308139,95706070037,5296328,40607732,A,1,2.0,JULIJANA BOTIC MD,BOTIC,JULIJANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308227,95707970058,3531552,27990790,A,1,2.0,DEJAN M NIKOLIC MD,NIKOLIC,DEJAN,MIODRAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308271,95708800059,2688335,14619640,A,1,2.0,BELMA KANTARDZIC MD,KANTARDZIC,BELMA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308288,95708820017,2845771,14619744,A,1,2.0,AZRA M SEHIC MD,SEHIC,AZRA,M,...,NaN,NaN,NaN,NaN,SEHIC,MEHMED,AZRA,NaN,NaN,NaN


In [98]:
with_source = pd.merge(with_polos, polo_addresses, left_on=['ME','POLO_COMM_ID'], right_on=['ME','comm_id'])

In [34]:
not_dea_aims = polo_addresses[~(polo_addresses.src_cat_code=='DEA       ')]

In [31]:
dea_dpc = pd.merge(nulls[nulls.ME.isin(with_polos_dpc.ME)], dea_aims, left_on=['ME','COMM_ID'], right_on=['ME','comm_id']).drop_duplicates('ME')

In [32]:
dea_ppd = pd.merge(nulls[nulls.ME.isin(with_polos.ME)], dea_aims, left_on=['ME','COMM_ID'], right_on=['ME','comm_id']).drop_duplicates('ME')

In [41]:
dea_ppd[~(dea_ppd.ME.isin(not_dea_aims.ME))].to_csv('../../Data/DEA/DEA_Only_AIMS.csv', index=False)

In [43]:
not_dea_aims[(not_dea_aims.ME.isin(dea_ppd.ME))].groupby('src_cat_code').count()

,entity_id,comm_id,ME,ENTITY_ID
src_cat_code,,,,
ACS,6133,6133,6133,6133
ACXIOMLODE,9,9,9,9
ACXIOMNCOA,1751,1751,1751,1751
ACXIOMPLUS,1,1,1,1
ADDR-VER,10,10,10,10
ADMIT-HOS,39,39,39,39
ADVR,1,1,1,1
AFFIL-GRP,336,336,336,336
AMA-ORG,113,113,113,113


In [17]:
pd.merge(nulls, polo_addresses, left_on=['ME','COMM_ID'], right_on=['ME','comm_id'])

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC,ENTITY_ID_x,entity_id,comm_id,src_cat_code,ENTITY_ID_y
0,1894036,00401530789,11103248,NaN,1406 College Dr,Mitchell Young M D,NaN,Texarkana,TX,75503,2005-12-16 20:27:40.957626,6454455,Office,1396131,1396131,6454455,ROSTER,1396131
1,1892959,00106930210,16561405,NaN,1215 7th St SE Ste 110,NaN,NaN,Decatur,AL,35601,2005-12-16 20:35:36.375262,9806586,Office,1333770,1333770,9806586,DEA,1333770
2,1891637,00102520483,16445710,NaN,8993 Woodhaven Dr,NaN,NaN,Pinson,AL,35126,2005-12-16 20:29:20.843635,7549648,Office,1242744,1242744,7549648,NCOA,1242744
3,1891638,00102530098,16395520,NaN,946 E Meighan Blvd,NaN,NaN,Gadsden,AL,35903,2005-12-16 20:28:04.602351,6606121,Office,1242861,1242861,6606121,UNKNOWN,1242861
4,1891638,00102530098,16395520,NaN,946 E Meighan Blvd,NaN,NaN,Gadsden,AL,35903,2005-12-16 20:28:04.602351,6606121,Office,1242861,1242861,6606121,LICENSURE,1242861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892727,10985070,02576112036,23389609,NaN,285 E State St,Ste 670,NaN,Columbus,OH,43215,2016-10-13 22:16:00.896023,18826564,Office,48274313,48274313,18826564,None,48274313
892728,10985071,02075130336,23389609,NaN,285 E State St,Ste 670,NaN,Columbus,OH,43215,2016-10-13 22:16:00.963538,18826564,Office,48274326,48274326,18826564,None,48274326
892729,10985074,03875151001,21944795,NaN,875 8th St NE,NaN,NaN,Massillon,OH,44646,2016-10-13 22:16:01.079380,15622353,Office,48274365,48274365,15622353,None,48274365
892730,10985077,04178160732,23389609,NaN,285 E State St,Ste 670,NaN,Columbus,OH,43215,2016-10-13 22:16:01.079878,18826564,Office,48274404,48274404,18826564,None,48274404


In [30]:
polo_addresses.head()

,entity_id,comm_id,src_cat_code,ME,ENTITY_ID
0,3028151,875974,UNKNOWN,01601760807,3028151
1,3035639,879852,MBSHP-MAIL,01601820052,3035639
2,3042932,883682,WHITE-MAIL,01601860623,3042932
3,3042932,883682,NPI,01601860623,3042932
4,3106619,903714,PPA,01602791196,3106619


In [36]:
polo_addresses = polo_addresses[polo_addresses.ME.isin(ppd.ME)]

In [62]:
polo_addresses = polo_addresses[polo_addresses.ME.isin(with_polos.ME)]

In [71]:
polo_addresses = polo_addresses.drop_duplicates(['ME','comm_id', 'src_cat_code'])
polo_addresses.drop_duplicates(['ME','comm_id'], keep=False)

,entity_id,comm_id,src_cat_code,ME,ENTITY_ID
0,3028151,875974,UNKNOWN,01601760807,3028151
1,3035639,879852,MBSHP-MAIL,01601820052,3035639
6,1240833,713865,PPA,00102490266,1240833
10,1243836,714512,PPA,00102540336,1243836
13,1244356,714633,UNKNOWN,00102550145,1244356
...,...,...,...,...,...
1773534,21475242,10704012,DEA,49537990120,21475242
1773536,43758282,28244225,MBSHP-OTHR,03901140226,43758282
1773537,45427560,27602124,PHONE-CALL,01106161085,45427560
1773538,39182685,9856658,PHONE-CALL,03840102172,39182685


In [112]:
with_source.drop_duplicates('ME', keep=False).groupby('src_cat_code').count()[0:50]

,ME,PARTY_ID,ENTITY_ID_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,entity_id,comm_id,ENTITY_ID_y
src_cat_code,,,,,,,,,,,,,,,,,,,,,
ACS,22273,22273,22273,22273,22273,22242,22273,22272,22273,18782,...,22068,6833,5869,6833,852,678,852,22273,22273,22273
ACSIMB-MMX,1352,1352,1352,1352,1352,1351,1352,1352,1352,1136,...,1352,336,284,336,46,34,46,1352,1352,1352
ACXIOM,11,11,11,11,11,11,11,11,11,11,...,11,4,3,4,0,0,0,11,11,11
ACXIOMLODE,168,168,168,168,168,152,168,168,168,144,...,168,51,44,51,10,7,10,168,168,168
ACXIOMNCOA,4782,4782,4782,4782,4782,4778,4782,4782,4782,3925,...,4774,1486,1258,1486,174,136,174,4782,4782,4782
ACXIOMPLUS,3,3,3,3,3,3,3,3,3,2,...,3,2,2,2,0,0,0,3,3,3
ADDR-VER,71,71,71,71,71,71,71,71,71,56,...,70,11,11,11,1,1,1,71,71,71
ADMIT-HOS,11612,11612,11612,11612,11612,11175,11612,11612,11612,9851,...,11343,4350,3688,4350,540,440,540,11612,11612,11612
AFFIL-GRP,8257,8257,8257,8257,8257,8032,8257,8257,8257,6952,...,8213,2716,2317,2716,361,294,361,8257,8257,8257


In [ ]:
polo_addresses.drop_duplicates(['ME','ENTITY_ID',''])

In [38]:
len(polo_addresses.drop_duplicates('ME'))

1078021

In [60]:
with_polos = ppd.dropna(subset=['POLO_MAILING_LINE_2'])

In [54]:
dpc = ppd[ppd.TOP_CD==20]

In [114]:
dea_edw = pd.read_csv('../../Data/DEA/DEA_Only.csv')

In [121]:
dea_edw.ME = use.fix_me(dea_edw.ME)

In [124]:
pd.merge(with_source, dea_edw, left_on=['ME','comm_id'], right_on=['ME','COMM_ID']).groupby('src_cat_code').count()

,ME,PARTY_ID_x,ENTITY_ID_x,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,...,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY_y,STATE_y,ZIP_y,FROM_DT_y,COMM_ID,PURPOSE_TYPE_DESC
src_cat_code,,,,,,,,,,,,,,,,,,,,,
DEA,147183,147183,147183,147183,147183,146245,147183,147180,147181,121619,...,147183,147183,118720,38184,147183,147183,147183,147183,147183,147183
LICENSURE,1634,1634,1634,1634,1634,1622,1634,1634,1634,1386,...,1634,1634,890,87,1634,1634,1634,1634,1634,1634
NPI,584,584,584,584,584,582,584,584,584,461,...,584,584,257,9,584,584,584,584,584,584
OTHER,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
PHONE-CALL,2,2,2,2,2,2,2,2,2,2,...,2,2,2,0,2,2,2,2,2,2


In [130]:
with_source[['ME','src_cat_code','comm_id']]

,ME,src_cat_code,comm_id
0,00102000013,NPI,28143514
1,00102000021,PHONE-CALL,11148513
2,00102000021,LICENSURE,11148513
3,00102000030,DEA,11727862
4,00102000048,LICENSURE,15446117
...,...,...,...
1231603,96501950014,DEA,11946751
1231604,96501960010,None,939632
1231605,96501960028,NPI,26371358
1231606,96501960052,PHNSURV,3537863


In [140]:
pd.merge(nulls, with_source[['ME','src_cat_code','comm_id']], left_on=['ME','COMM_ID'], right_on=['ME','comm_id']).groupby('src_cat_code').count()[0:50]

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC,ENTITY_ID,comm_id
src_cat_code,,,,,,,,,,,,,,,
ACS,27399,27399,27399,0,27399,15055,12,27399,27399,27399,27399,27399,27399,27399,27399
ACXIOM,11,11,11,0,11,4,0,11,11,11,11,11,11,11,11
ACXIOMLODE,197,197,197,0,197,57,0,197,197,197,197,197,197,197,197
ACXIOMNCOA,5168,5168,5168,0,5168,2529,2,5168,5168,5168,5168,5168,5168,5168,5168
ACXIOMPLUS,4,4,4,0,4,2,0,4,4,4,4,4,4,4,4
ADDR-VER,92,92,92,0,92,46,3,92,92,92,92,92,92,92,92
ADMIT-HOS,9414,9414,9414,0,9414,35,9,9414,9414,9414,9414,9414,9414,9414,9414
ADVR,2,2,2,0,2,0,0,2,2,2,2,2,2,2,2
AFFIL-GRP,10314,10314,10314,0,10314,2809,30,10314,10314,10314,10314,10314,10314,10314,10314


In [135]:
len(nulls)

550616

In [6]:
query_3 = \
    f'''
    SELECT
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE,
    p.addr_line2, 
    p.addr_line1, 
    p.city_cd, 
    p.state_cd, 
    p.zip
    FROM
    informix.ENTITY_COMM_AT U join informix.POST_ADDR_AT P on U.COMM_ID = P.COMM_ID
    WHERE
    U.SRC_CAT_CODE = 'CRED-ADDR'
    ;
    '''
cred_addresses = pd.read_sql(con=informix, sql=query_3)
cred_addresses.head()

,entity_id,comm_id,src_cat_code,addr_line2,addr_line1,city_cd,state_cd,zip
0,1234658,26789742,CRED-ADDR,1653 Temple Ave N,P O Drawer 710,Fayette,AL,35555
1,1234671,26577992,CRED-ADDR,3719 Dauphin St Frnt,,Mobile,AL,36608
2,1234671,26807023,CRED-ADDR,2451 University Hospital Dr,Mastin Bldg - 6th Floor,Mobile,AL,36617
3,1234892,26772747,CRED-ADDR,1613 N McKenzie St,,Foley,AL,36535
4,1234892,26789730,CRED-ADDR,1653 Temple Ave N,P O Drawer 710,Fayette,AL,35555


In [21]:
ENTITY_TO_ME['entity_id'] = ENTITY_TO_ME.ENTITY_ID.astype(int)

In [12]:
QUERY = \
        """
        SELECT DISTINCT 
        M.ME, E.ENTITY_ID 
        FROM 
        AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E 
        WHERE 
        M.PARTY_ID=E.PARTY_ID
        """
    #Execute queries
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)

ENTITY_TO_ME = pd.read_sql(con=AMAEDW, sql=QUERY)
ENTITY_TO_ME['entity_id'] = ENTITY_TO_ME.ENTITY_ID.astype(int)
load = pd.merge(nat_down_addresses, ENTITY_TO_ME, on='entity_id')

In [26]:
load = pd.merge(nat_down_addresses, ENTITY_TO_ME, on='entity_id')

In [25]:
phones = pd.read_csv('../../Data/DataGov/Phone_Sample.csv')

In [34]:
phones['zip'] = [use.fix_zipcode(x) for x in phones.addr_zip]
phones['ME'] = use.fix_me(phones.ME)

In [36]:
phones[['ME','addr_line_2', 'addr_line_1','addr_city', 'addr_state', 'zip']]

,ME,addr_line_2,addr_line_1,addr_city,addr_state,zip
0,49575000010,SUITE 4E,951 NW 13TH ST,BOCA RATON,FL,33486
1,49575000010,SUITE 120,840 US HWY 1,NORTH PALM BEACH,FL,33408
2,02106050565,SUITE 4000,7777 HENNESSY BLVD,BATON ROUGE,LA,70808
3,02106050565,SUITE 3000,7777 HENNESSY BLVD,BATON ROUGE,LA,70808
4,04113040195,10A,980 BEAVER GRADE RD,CORAOPOLIS,PA,15108
...,...,...,...,...,...,...
447247,05104181637,SANTA CLAVA VLY MED,751 S BASCOM AVE,SAN JOSE,CA,95128
447248,01104140765,E 101,6798 CROSSWINDS DR,SAINT PETERSBURG,FL,33710
447249,66501150721,SUITE 230,2409 N PATTERSON ST,VALDOSTA,GA,31602
447250,06509140021,FL 4,4 WESTCHESTER PARK DR,WHITE PLAINS,NY,10604


In [40]:
load[['ME','addr_line_2', 'addr_line_1','addr_city', 'addr_state', 'zip']]

,ME,addr_line_2,addr_line_1,addr_city,addr_state,zip
0,49575000010,STE 4E,951 NW 13TH ST,BOCA RATON,FL,33486
1,49575000010,STE 120,840 US HIGHWAY 1,NORTH PALM BEACH,FL,33408
2,49575000010,,2330 S CONGRESS AVE,PALM SPRINGS,FL,33406
3,00102000048,,1111 E MCDOWELL RD,PHOENIX,AZ,85006
4,00102000048,,14502 W MEEKER BLVD,SUN CITY,AZ,85375
...,...,...,...,...,...,...
1330849,02775150672,STE 2220,1105 CENTRAL EXPY N,ALLEN,TX,75013
1330850,02775150672,STE 500,4521 MEDICAL CTR DR,MCKINNEY,TX,75069
1330851,02775150672,STE 205,5425 W SPRING CREEK PKWY,PLANO,TX,75024
1330852,02775150672,STE 500,4521 MEDICAL CTR DR,MCKINNEY,TX,75069


In [48]:
load['addr_line_1'] = [x.upper().strip() for x in load.addr_line2]
load['addr_line_2'] = [x.upper().strip() for x in load.addr_line1]
load['addr_city'] = [x.upper().strip() for x in load.city_cd]
load['zip'] = [x.strip() for x in load.zip]
load['addr_state'] = load.state_cd

AttributeError: 'NoneType' object has no attribute 'strip'

In [69]:
all_this = pd.merge(load, phones, on=['ME','addr_line_1','addr_city', 'addr_state']).drop_duplicates()

In [68]:
[['ME','entity_id','comm_id','src_cat_code','phone_number']].to_csv('../../PhoneAppend/National_Downloadable_File_Load_2021-11-15.csv', index=False)

,ME,entity_id,comm_id,src_cat_code,phone_number
0,49575000010,21918360,26491704,NAT-DOWN,5619239906
1,00102000064,1234645,27553719,NAT-DOWN,2052217099
2,00102000072,1234658,27544478,NAT-DOWN,2053457000
3,00102000196,1234814,27650757,NAT-DOWN,2514458242
4,00102000196,1234814,27650758,NAT-DOWN,2514458242
...,...,...,...,...,...
382811,04177121881,65910442,28156187,NAT-DOWN,2153456050
382812,04177112173,66081132,28156398,NAT-DOWN,5705016730
382813,04178161682,67154204,28156537,NAT-DOWN,4356281641
382814,02775150672,67582177,28156576,NAT-DOWN,9725628383


In [86]:
all_this.to_csv('../../Data/DataGov/Load_With_Phones', index=False)

In [ ]:
412-2624-91
225-766-744
561-9239-91

In [66]:
phones['phone_number'] = [use.fix_phone(x) for x in phones.phone]

In [ ]:
def get_present_employment():
    present_employment_key = {
        11: 'Self-Employed Solo Practice',
        13: 'Two Physician Practice-Full Or Part Owner',
        21: 'Other-Patient Care',
        22: 'Locum Tenens',
        30: 'Group Practice',
        35: 'HMO',
        40: 'Medical School',
        50: 'Non-Government Hospital',
        63: 'City/County/State Government-Hospital',
        64: 'City/County/State Government-Other Than Hospital',
        81: 'Federal Government-Hospital/Army',
        82: 'Federal Government-Hospital/Navy',
        83: 'Federal Government-Hospital/Air Force',
        84: 'Federal Government-Hospital/Usphs',
        85: 'Federal Government-Hospital/Vet Admin',
        86: 'Federal Government-Hospital/Other Agency',
        101: 'Other/Non-Patient Care',
        110: 'No Classification'
    }
    return present_employment_key

def humach_samplify(data):
    present_employment_key = get_present_employment()
    data['DESCRIPTION'] = [present_employment_key[x] for x in data.PE_CD]
    data = data[data.TELEPHONE_NUMBER!='None']
    new_columns = {
        'ADDR_1_POLO':'POLO_MAILING_LINE_2',
        'ADDR_2_POLO':'POLO_MAILING_LINE_1',
        'CITY_POLO':'POLO_CITY',
        'STATE_POLO':'POLO_STATE',
        'ZIP_POLO':'POLO_ZIP'}
    humach_columns = [
        'ME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'LAST_NAME',
        'SUFFIX',
        'ADDR_2_POLO',
        'ADDR_1_POLO',
        'CITY_POLO',
        'STATE_POLO',
        'ZIP_POLO',
        'TELEPHONE_NUMBER',
        'PRIM_SPEC_CD',
        'DESCRIPTION',
        'PE_CD',
        'FAX_NUMBER'
    ]
    humach_data = data[humach_columns].rename(columns = new_columns)
    return humach_data

In [ ]:
def clean_file(all_that, today):
    '''Clean and format data'''
    LOGGER.info('Cleaning file')
    all_that = all_that.drop_duplicates(['ME'])
    all_that = remove_whitespaces(all_that)
    all_that['addr_plus4'] = ''
    all_that['addr_zip'] = fix_zipcodes(all_that.ZIP_POLO)
    all_that['addr_country'] = ''
    all_that['ADDR_2_POLO'] = ['' if x=='None' else x for x in all_that.ADDR_2_POLO]
    all_that['ADDR_3_POLO'] = ['' if x=='None' else x for x in all_that.ADDR_3_POLO]
    all_that['usage'] = 'PP'
    all_that['load_type_ind'] = 'R'
    all_that['addr_type'] = 'OF'
    all_that['source'] = 'POLO-PPD'
    all_that['source_dtm'] = today
    col = get_col()
    all_that = all_that[['ENTITY',
                         'ME',
                         'COMM_ID',
                         'usage',
                         'load_type_ind',
                         'addr_type',
                         'ADDR_1_POLO',
                         'ADDR_2_POLO',
                         'ADDR_3_POLO',
                         'CITY_POLO',
                         'STATE_POLO',
                         'addr_zip',
                         'addr_plus4',
                         'addr_country',
                         'source',
                         'source_dtm',
                         ]].rename(columns=col)
    return all_that

In [78]:
all_this.columns

Index(['entity_id', 'comm_id', 'src_cat_code', 'addr_line2', 'addr_line1',
       'city_cd', 'state_cd', 'zip_x', 'ME', 'ENTITY_ID', 'addr_line_1',
       'addr_line_2_x', 'addr_city', 'addr_state', 'me#', 'usage', 'load_type',
       'addr_type', 'addr_line_2_y', 'addr_line_3', 'addr_zip', 'addr_plus4',
       'addr_country', 'source', 'source_dtm', 'adrs_id', 'phone', 'zip_y',
       'phone_number'],
      dtype='object')

In [81]:
for row in all_this.itertuples():
    if row.zip_y not in row.zip_x:
        print(f'{row.addr_line_1} {row.addr_city}')
        print(f'{row.addr_line_2_y} is not {row.addr_line_2_x}')
        print(f'{row.zip_y} is not {row.zip_x}')

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
102 MICAH WAY SCOTTSBORO
SUITE 1107 is not STE 1107
35768 is not 35769     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE 102
36301 is not 36305     
1088 9TH AVE SW BESSEMER
SUITE 104 is not STE 104
35023 is not 35022     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE 102
36301 is not 36305     
1717 N E ST PENSACOLA
SUITE 308 is not STE 308
32505 is not 32501     
1717 N E ST PENSACOLA
SUITE 308 is not STE 308
32505 is not 32501     
1088 9TH AVE SW BESSEMER
SUITE 104 is not STE 104
35023 is not 35022     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGE

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
1600 DIVISADERO ST SAN FRANCISCO
MCB302 is not MCB302
94115 is not 94143     
2001 W 68TH ST HIALEAH
PALMETTO GENERAL HOPSITAL is not PALMETTO GENERAL HOPSITAL
33176 is not 33016     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2750 SW 37TH AVE MIAMI
SUITE 100 is not STE 100
33134 is not 33133

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2200 ADA AVE CONWAY
SUITE 301 is not STE 301
72032 is not 72034     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
U

32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZ

...

45458 is not 45459     
1 W RIDGEWOOD AVE PARAMUS
SUITE 211 HACKENSACK UNIVERSITY MEDICAL GROUP is not SUITE 211 HACKENSACK UNIVERSITY MEDICAL
07630 is not 07652     
225 WILLIAMSON ST ELIZABETH
TRINITAS HOSPITAL is not TRINITAS HOSPITAL
07207 is not 07202     
210 E 64TH ST NEW YORK
4TH FLOOR is not 4TH FL
11065 is not 10065     
525 E 68TH ST NEW YORK
PULMONARY DISEASE is not PULMONARY DISEASE
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
420 5TH AVE W HENDERSONVILLE
SUITE 300 is not STE 300
28793 is not 28739     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1989 MIAMISBURG CENTERVILLE RD DAYTON
SUITE 204 is not STE 204
45342 is not 45459     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1120 SE CARY PKWY CARY
SUITE 204 is not STE 204
27511 is not

28210 is not 28287     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
587 BIELENBERG DR WOODBURY
SUITE 200 is not STE 200
55110 is not 55125     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
525 E 68TH ST NEW YORK
PULMONARY DISEASE is not PULMONARY DISEASE
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1 W RIDGEWOOD AVE PARAMUS
SUITE 301 is not STE 301
07662 is not 07652     
587 BIELENBERG DR WOODBURY
SUITE 200 is not STE 200
55110 is not 55125     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
525 E 68TH ST NEW YORK
GYNECOLOGIC ONCOLOGY is not GYNECOLOGIC ONCOLOGY
10021 is n

SUITE 200 is not STE 200
37849 is not 37909     
130 FISHER RD BARRE
CENTRAL VERMONT is not 
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
4120 W MEMORIAL RD OKLAHOMA CITY
SUITE 300 is not STE 300
76312 is not 73120     
110 W 1325 N CEDAR CITY
SUITE 300 is not STE 300
84720 is not 84721     
110 W 1325 N CEDAR CITY
SUITE 300 is not STE 300
84720 is not 84721     
400 W LYNDON B JOHNSON FWY IRVING
SUITE 350 is not STE 350
75069 is not 75063     
985 BERKSHIRE BLVD WYOMISSING
SUITE 101 is not STE 101
19601 is not 19610     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not PATHOLOGY CLINIC
57104 is not 57105     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not 
57104 is not 57105     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not CHILD SPEC CLIN
57104 is not 57105     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is 

3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3712 W 7TH ST FORT WORTH
MONTICELLO DIAGNOST is not MONTICELLO DIAGNOST
76104 is not 76107     
110 W 1325 N CEDAR CITY
SUITE 200 is not STE 200
84720 is not 84721     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not

2810 ASHLEY PHOSPHATE RD CHARLESTON
SUITE B 11 is not STE B11
29425 is not 29418     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
16463 BOONES FERRY RD LAKE OSWEGO
SUITE 300 is not STE 300
97305 is not 97035     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
1330 TAYLOR AT MARION ST COLUMBIA
SUITE 100 is not 
29201 is not 29220     
5126 W DAYBREAK PKWY SOUTH JORDAN
UUHC SOUTH JORDAN HEALTH CENTER is not UUHC SOUTH JORDAN HEALTH CENTER
84095 is not 84009     
2810 ASHLEY

In [ ]:
['entity_id', 'me#','comm_id', 'src_cat_code', 'addr_line2', 'addr_line1',
       'city_cd', 'state_cd', 'zip_x', 'ME', 'ENTITY_ID', 'addr_line_1',
       'addr_line_2_x', 'addr_city', 'addr_state',  'usage', 'load_type',
       'addr_type', 'addr_line_2_y', 'addr_line_3', 'addr_zip', 'addr_plus4',
       'addr_country', 'source', 'source_dtm', 'adrs_id', 'phone', 'zip_y']

In [ ]:
[['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm', 'phone_number']]

In [82]:
len(cred_addresses)

312618

In [84]:
cred_addresses = pd.merge(cred_addresses, ENTITY_TO_ME, on='entity_id')
cred_addresses.to_csv('../../Data/Credentialling/Data_Load.csv', index=False)